In [1]:
import numpy as np
import pandas as pd 
from tqdm import tqdm

ImportError: No module named 'tqdm'

# Kernel Methods for Machine Learning

## Data Loading and exploration

In [ ]:
# Train features
df_Xtr0 = pd.read_csv("Data/Xtr0.csv")
df_Xtr1 = pd.read_csv("Data/Xtr1.csv")
df_Xtr2 = pd.read_csv("Data/Xtr2.csv")

df_Xtr0_mat100 = pd.read_csv("Data/Xtr0_mat100.csv", header=None, sep=' ')
df_Xtr1_mat100 = pd.read_csv("Data/Xtr1_mat100.csv", header=None, sep=' ')
df_Xtr2_mat100 = pd.read_csv("Data/Xtr2_mat100.csv", header=None, sep=' ')

# Train labels
df_Ytr0 = pd.read_csv("Data/Ytr0.csv")
df_Ytr1 = pd.read_csv("Data/Ytr1.csv")
df_Ytr2 = pd.read_csv("Data/Ytr2.csv")

# Test features
df_Xte0 = pd.read_csv("Data/Xte0.csv")
df_Xte1 = pd.read_csv("Data/Xte1.csv")
df_Xte2 = pd.read_csv("Data/Xte2.csv")

df_Xte0_mat100 = pd.read_csv("Data/Xte0_mat100.csv", header=None, sep=' ')
df_Xte1_mat100 = pd.read_csv("Data/Xte1_mat100.csv", header=None, sep=' ')
df_Xte2_mat100 = pd.read_csv("Data/Xte2_mat100.csv", header=None, sep=' ')

## 2. Gaussian Kernel

### 2.1 Define the kernel

In [ ]:
## We need to parallize its computation
def GaussKernel(X1, X2, sigma = 1):
    n, m = X1.shape[0], X2.shape[0]
    K = np.zeros((n,m))
    
    for i in range(n):
        for j in range(m):
            K[i,j] = np.exp(-((np.linalg.norm(X1[i]-X2[j]))**2)/(2*sigma**2))
    return K

We compute the Kernel matrix for each of the tree train sets and we save them in *Kernel_Matrix* directory

In [ ]:
# Tranforming into numpy.arrays -- train
Xtr0_mat100 = np.array(df_Xtr0_mat100)
Xtr1_mat100 = np.array(df_Xtr1_mat100)
Xtr2_mat100 = np.array(df_Xtr2_mat100)

# Tranforming into numpy.arrays -- test
Xte0_mat100 = np.array(df_Xte0_mat100)
Xte1_mat100 = np.array(df_Xte1_mat100)
Xte2_mat100 = np.array(df_Xte2_mat100)

# Transforming the labels into numpy.arrays 
y0 = np.array(df_Ytr0)[:,1]
y1 = np.array(df_Ytr1)[:,1]
y2 = np.array(df_Ytr2)[:,1]

# We should parallelize this computation
K_Xtr0 = GaussKernel(Xtr0_mat100, Xtr0_mat100)
np.save("Kernel_Matrix/gaussian_kernel_Xtr0.npy",K_Xtr0)

K_Xtr1 = GaussKernel(Xtr1_mat100, Xtr1_mat100)
np.save("Kernel_Matrix/gaussian_kernel_Xtr1.npy",K_Xtr1)

K_Xtr2 = GaussKernel(Xtr2_mat100, Xtr2_mat100)
np.save("Kernel_Matrix/gaussian_kernel_Xtr2.npy",K_Xtr2)

### 2.2. Implement SVM with gaussian kernel

We solve the optimization problem $$\left\{\begin{matrix}
\underset{\alpha \in \mathbb{R}^{n}}{\text{max}} \hspace{0.1cm} 2\alpha^{T}y - \alpha^{T}K\alpha \\ 0 \leq y_i\alpha_i \leq \frac{1}{2\lambda n}, \hspace{0.5cm} \text{for} \hspace{0.3cm} i = 0...n
\end{matrix}\right. \Leftrightarrow \left\{\begin{matrix}
\underset{\alpha \in \mathbb{R}^{n}}{\text{min}} \hspace{0.1cm} \frac{1}{2}\alpha^{T}P\alpha + q^{t}\alpha  \\ G\alpha \leq h
\end{matrix}\right.   $$
Where $\tilde{P} = K$, $q = -y$, $G =\binom{\text{Diag}(y)}{-\text{Diag}(y)} $ and $h=\binom{\frac{1}{2\lambda n}\mathcal{1}}{0}$

In [ ]:
import cvxopt
from cvxopt import matrix

def solve_dual_SVM(K,y, lambda_ = 1):
    n = K.shape[0] 
    G = np.vstack((np.diag(y),-np.diag(y)))
    h = np.vstack(((1/(2*lambda_*n))*np.ones((n,1)),np.zeros((n,1))))

    P = K
    q = -y.reshape(-1,1)
    #P = .5 * (P + P.T)  # make sure P is symmetric
    args = [matrix(P), matrix(q)]

    args.extend([matrix(G), matrix(h)])

    sol = cvxopt.solvers.qp(*args) 

    return np.array(sol['x']).reshape((P.shape[1],))
    


In [ ]:
alpha_star0 = solve_dual_SVM(K_Xtr0,y0)
alpha_star1 = solve_dual_SVM(K_Xtr1,y1)
alpha_star2 = solve_dual_SVM(K_Xtr2,y2)

### 2.3. Predictions on test set  

In [ ]:
# We should parallelize this computation
K_Xte0 = GaussKernel(Xtr0_mat100, Xte0_mat100)
np.save("Kernel_Matrix/gaussian_kernel_Xte0.npy",K_Xte0)

K_Xte1 = GaussKernel(Xtr1_mat100, Xte1_mat100)
np.save("Kernel_Matrix/gaussian_kernel_Xte1.npy",K_Xte1)

K_Xte2 = GaussKernel(Xtr2_mat100, Xte2_mat100)
np.save("Kernel_Matrix/gaussian_kernel_Xte2.npy",K_Xte2)

In [ ]:
prediction0 = alpha_star0.reshape(-1,1).T.dot(K_Xte0)
prediction0[prediction0>0] = 1
prediction0[prediction0 <0] = 0

prediction1 = alpha_star1.reshape(-1,1).T.dot(K_Xte1)
prediction1[prediction1>0] = 1
prediction1[prediction1 <0] = 0

prediction2 = alpha_star2.reshape(-1,1).T.dot(K_Xte2)
prediction2[prediction2>0] = 1
prediction2[prediction2 <0] = 0

### 1.4 Sickit-learn

In [ ]:
from sklearn.svm import SVC

clf = SVC(gamma='auto')
clf.fit(Xtr0_mat100, y0)
predciton0_sk = clf.predict(Xte0_mat100)

clf = SVC(gamma='auto')
clf.fit(Xtr1_mat100, y1)
predciton1_sk = clf.predict(Xte1_mat100)

clf = SVC(gamma='auto')
clf.fit(Xtr2_mat100, y2)
predciton2_sk = clf.predict(Xte2_mat100)

## 1. Spectrum Kernel

### 1.1. Define the kernel

In [ ]:
def getSubString(mString, spectrum):
    
    dictionnary = {}
    for i in range(len(mString)-spectrum+1):
        if mString[i:i+spectrum] in dictionnary:
            dictionnary[mString[i:i+spectrum]] += 1
        else:
            dictionnary[mString[i:i+spectrum]] = 1
    return dictionnary

def SpectrumKernelFunction(mString1, mString2, spectrum):
    dictionnary1 = getSubString(mString1, spectrum)
    dictionnary2 = getSubString(mString2, spectrum)
    
    kernel = 0
    for i in dictionnary1:
        if i in dictionnary2:
            kernel += dictionnary1[i] * dictionnary2[i]
    return kernel

## We should improve this function to take less time
def SpectrumKernelMatrix_train(serie,spectrum):
    n = serie.shape[0]
    K = np.zeros((n,n))
    for i,seq1 in enumerate(serie):
        for j,seq2 in enumerate(serie):
            if i <= j :
                K[i,j] = SpectrumKernelFunction(seq1, seq2, spectrum)
                K[j,i] = K[i,j]
    return(K)

def SpectrumKernelMatrix_test(serie_train, serie_test, spectrum):
    n = serie_train.shape[0]
    m = serie_test.shape[0]
    K = np.zeros((n,m))
    for i,seq1 in enumerate(serie_test):
        for j,seq2 in enumerate(serie_train):
            K[j,i] = SpectrumKernelFunction(seq1, seq2, spectrum)
    return(K)
    

We compute the Kernel matrix for each of the tree train sets and we save them in *Kernel_Matrix* directory

In [ ]:
# We should parallelize this computation
K_Xtr0 = SpectrumKernelMatrix_train(df_Xtr0['seq'],spectrum=3)
np.save("Kernel_Matrix/spectrum_kernel_Xtr0.npy",K_Xtr0)

K_Xtr1 = SpectrumKernelMatrix_train(df_Xtr1['seq'],spectrum=3)
np.save("Kernel_Matrix/spectrum_kernel_Xtr1.npy",K_Xtr1)

K_Xtr2 = SpectrumKernelMatrix_train(df_Xtr2['seq'],spectrum=3)
np.save("Kernel_Matrix/spectrum_kernel_Xtr2.npy",K_Xtr2)


We compute the Kernel matrix for each of the tree test sets and we save them in *Kernel_Matrix* directory

In [ ]:
# We should parallelize this computation
K_Xte0 = SpectrumKernelMatrix_train(df_Xtr0['seq'],df_Xte0['seq'],spectrum=3)
np.save("Kernel_Matrix/spectrum_kernel_Xte0.npy",K_Xte0)

K_Xte1 = SpectrumKernelMatrix_train(df_Xtr1['seq'],df_Xte1['seq'],spectrum=3)
np.save("Kernel_Matrix/spectrum_kernel_Xte1.npy",K_Xte1)

K_Xte2 = SpectrumKernelMatrix_train(df_Xtr2['seq'],df_Xte2['seq'],spectrum=3)
np.save("Kernel_Matrix/spectrum_kernel_Xte2.npy",K_Xte2)

### 1.2. Solve the standard weighted kernel logisitc regression (WKLR) problem

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

### We need to improve this ####
def sqrtMatrix(W):
    # To compute the square root of a symetric positive matrix
    D,V = np.linalg.eig(W)
    return np.dot(np.dot(V,np.diag(np.sqrt(D))),np.linalg.inv(V))

def solveWKRR(K,W,z,lambda_):
    n = K.shape[0]
    W_sqrt = np.real(sqrtMatrix(W))
    
    temp = np.dot(np.dot(W_sqrt,K),W_sqrt) +  n*lambda_*np.eye(n)
    return  np.dot(W,np.linalg.solve(temp,np.dot(W_sqrt,z)))

def solveKLR(K,y,alpha0,lambda_ = 1,itermax = 30, eps =1e-6):
    n = K.shape[0]
    
    iter_ = 0
    last_alpha = 10*alpha0 + np.ones(alpha0.shape)
    alpha = alpha0
    
    while (iter_< itermax) and (np.linalg.norm(last_alpha-alpha)>eps) :         
        print(iter_,np.linalg.norm(last_alpha-alpha))
        last_alpha = alpha
        m = np.dot(K,alpha)
        P = np.zeros((n,1))
        W = np.zeros((n,n))
        z = np.zeros((n,1))
        for i in range(n):
            P[i,0] = -sigmoid(-y[i]*m[i])
            W[i,i] = sigmoid(m[i])*sigmoid(-m[i])
            z[i,0] = m[i] - (P[i,0]*y[i])/W[i,i]
        alpha = solveWKRR(K,W,z,lambda_)
        iter_ = iter_ +1
        
      
    return alpha        

In [ ]:
K = np.load("Kernel_Matrix/spectrum_kernel_Xtr0.npy")
y = df_Ytr0['Bound']
n = y.shape[0]
alpha0 = np.zeros(n) 

In [ ]:
alpha = solveKLR(K,y,alpha0) 

In [ ]:
def sign(x):
    y = x
    n = x.shape[0]
    for i in range(n):
        if x[i,0] > 0:
            y[i,0] = 1
        else:
            y[i,0] = 0
    return y

print(alpha,np.dot(K,alpha),y)

In [ ]:
print(K)

### 1.3 Results